#Belajar Pyspark - GroupBy dan Agregasi

Bagi anda yang familiar dengan SQL atau pandas, `groupBy` pada pyspark dataframe serupa dengan klausa `GROUP BY` pada SQL ataupun `groupby()` pada pandas dataframe. Fungsinya adalah untuk mengelompokkan data berdasar satu atau beberapa nilai kolom yang sama.


In [1]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=c5d80e244b1ade03e806bf9457e504178adda76a66a661c9c3a207e78e12444c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [17]:
spark = SparkSession.builder.appName("Belajar PySpark - Grouping dan Agregasi").getOrCreate()

Sebelumnya kita buat DataFrame dari sebuah python list, lalu kita akan lakukan transformasi terhadap DataFrame tersebut.


In [18]:
data = [['Agus','Fisika',1,130,100],['Windy','Fisika',2,200,150],
        ['Budi','Biologi',1,200,150],['Dina','Fisika',2,150,200],
        ['Bayu','Fisika',1,150,200],['Dedi','Biologi',2,50,150]]

kolom = ["nama","jurusan","semester","nilai1","nilai2"]

df = spark.createDataFrame(data,kolom)

df.show()

+-----+-------+--------+------+------+
| nama|jurusan|semester|nilai1|nilai2|
+-----+-------+--------+------+------+
| Agus| Fisika|       1|   130|   100|
|Windy| Fisika|       2|   200|   150|
| Budi|Biologi|       1|   200|   150|
| Dina| Fisika|       2|   150|   200|
| Bayu| Fisika|       1|   150|   200|
| Dedi|Biologi|       2|    50|   150|
+-----+-------+--------+------+------+



Fungsi groupBy mengembalikan object bertipe pyspark.sql.group.GroupedData. Object ini memiliki beberapa fungsi yang diperlukan untuk melakukan agregasi, yaitu : `sum, avg, max, min, count`, dan `agg`.


In [19]:
result = df.groupBy("jurusan")
type(result)

pyspark.sql.group.GroupedData

##Agregasi dengan sum, avg, min dan max

Jika tidak ditentukan kolom mana yang akan diagregasi (tanpa parameter), maka akan dilakukan fungsi agregasi yang sama untuk semua kolom numerik


In [20]:
result.avg().show()

+-------+-------------+-----------+-----------+
|jurusan|avg(semester)|avg(nilai1)|avg(nilai2)|
+-------+-------------+-----------+-----------+
| Fisika|          1.5|      157.5|      162.5|
|Biologi|          1.5|      125.0|      150.0|
+-------+-------------+-----------+-----------+



Kita bisa memilih satu atau beberapa kolom yang akan diagregasi dengan menuliskan nama kolom sebagai parameter

In [36]:
df.groupBy("jurusan","semester").avg("nilai1").show()

+-------+--------+-----------+
|jurusan|semester|avg(nilai1)|
+-------+--------+-----------+
|Biologi|       1|      200.0|
| Fisika|       1|      140.0|
| Fisika|       2|      175.0|
|Biologi|       2|       50.0|
+-------+--------+-----------+



###Agregat multiple kolom

In [39]:
df.groupBy("jurusan","semester").min("nilai1","nilai2").show()
df.groupBy("jurusan","semester").max("nilai1","nilai2").show()

+-------+--------+-----------+-----------+
|jurusan|semester|min(nilai1)|min(nilai2)|
+-------+--------+-----------+-----------+
|Biologi|       1|        200|        150|
| Fisika|       1|        130|        100|
| Fisika|       2|        150|        150|
|Biologi|       2|         50|        150|
+-------+--------+-----------+-----------+

+-------+--------+-----------+-----------+
|jurusan|semester|max(nilai1)|max(nilai2)|
+-------+--------+-----------+-----------+
|Biologi|       1|        200|        150|
| Fisika|       1|        150|        200|
| Fisika|       2|        200|        200|
|Biologi|       2|         50|        150|
+-------+--------+-----------+-----------+



###Agregasi dengan `Count()`

Berbeda dengan fungsi lainnya, `count()` tidak menerima parameter apapun. Fungsi `count()` mengembalikan jumlah record tiap group


In [9]:
df.groupBy("jurusan").count().show()

+-------+-----+
|jurusan|count|
+-------+-----+
| Fisika|    4|
|Biologi|    2|
+-------+-----+



##agregasi dengan fungsi `agg()`

Fungsi agg menerima parameter berupa fungsi-fungsi agregasi dari [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#aggregate-functions) atau dictionary mapping `{namakolom : fungsi agregasi}`.

Kelebihan menggunakan `agg()` diantaranya adalah :
pilihan fungsi agregasi yang luas,
dapat menggunakan beberapa fungsi agregasi yang berbeda dalam satu grouping, dan
dapat menggunakan alias untuk menamai kolom hasil agregasi.


In [13]:
df.groupBy("jurusan").agg(F.avg("nilai2")).show()

+-------+-----------+
|jurusan|sum(nilai2)|
+-------+-----------+
| Fisika|        650|
|Biologi|        300|
+-------+-----------+



###Multiple aggregation

In [15]:
df.groupBy("jurusan").agg(F.min("nilai1"),
                          F.max("nilai1")).show()

+-------+-----------+-----------+
|jurusan|min(nilai1)|max(nilai1)|
+-------+-----------+-----------+
| Fisika|        130|        200|
|Biologi|         50|        200|
+-------+-----------+-----------+



###Menggunakan alias

In [46]:
df.groupBy("jurusan") \
    .agg(F.min("nilai1").alias("min_nilai1"),
         F.max("nilai1").alias("max_nilai1"),
         F.median("nilai1").alias("median_nilai1"),
         F.count("nilai1").alias("jumlah_siswa")).show()

+-------+----------+----------+-------------+------------+
|jurusan|min_nilai1|max_nilai1|median_nilai1|jumlah_siswa|
+-------+----------+----------+-------------+------------+
| Fisika|       130|       200|        150.0|           4|
|Biologi|        50|       200|        125.0|           2|
+-------+----------+----------+-------------+------------+



###`agg()` dengan dictionary

Meskipun cukup jarang digunakan karena relatif sulit dibaca dan kurang fleksibel, kita dapat menggunakan dictionary


In [43]:
df.groupBy().agg(
    {"*": "count","nilai1":"min","nilai2":"max"}).show()

+-----------+-----------+--------+
|max(nilai2)|min(nilai1)|count(1)|
+-----------+-----------+--------+
|        200|         50|       6|
+-----------+-----------+--------+



##Agregasi dengan expr()

Selain menggunakan fungsi-fungsi agregasi, kita juga dapat menggunakan fungsi expr(). Fungsi ini bermanfaat jika kita ingin melakukan pemrosesan kolom sebelum diagregasi. Misalnya seperti contoh di bawah ini


In [47]:
df.groupBy("jurusan", "semester").\
    agg(F.expr("avg(round((nilai1 + nilai2)/5)) as rerata_total")).show()

+-------+--------+------------+
|jurusan|semester|rerata_total|
+-------+--------+------------+
|Biologi|       1|        70.0|
| Fisika|       1|        58.0|
| Fisika|       2|        70.0|
|Biologi|       2|        40.0|
+-------+--------+------------+

